In [338]:
import time
import json
import re
import os 
import tensorflow as tf
import pandas as pd
import numpy as np
from itertools import compress
import matplotlib.pyplot as plt


In [339]:
def json_to_csv(directory, fileNames, createSample=False):
    """
    json_to_csv: loops through specified JSON files and converts them to csv files.
                 option to also create a sample csv, which uses np.random.seed 9001 to create a sample dataset with 10% of the observations
    
                 pandas has a read_json function, but returns a 'Trailing data error' when working with these specific files
                 
                 
    Inputs: -directory of JSON files
            -list of JSON filenames
            -createSample flag
            
    """
    
    start = time.time()

    jsonData = []

    for fileName in fileNames:
        with open(directory + fileName,  encoding="utf8") as file:
            print('{0} opened'.format(fileName))
            for line in file:
                #I use an rstrip here because some of the files have trailing blank spaces
                jsonData.append(json.loads(line.rstrip()))
        
        df = pd.DataFrame.from_dict(jsonData)
        
        csvFileName = fileName[:len(fileName)-5] + '.csv'
        
        df.to_csv(directory + csvFileName)
        print('{0} created'.format(csvFileName))
        
        
        if createSample:
            np.random.seed(9001)
            msk = np.random.rand(len(df)) <= 0.1
            sample = df[msk]
            
            csvSampleFileName = fileName[:len(fileName)-5] + '_sample.csv'
            
            sample.to_csv(directory + csvSampleFileName)
            print('{0} created'.format(csvSampleFileName))
        
    print('This function took {} minutes to run'.format((time.time()-start)/60))
    

In [340]:
# fileNameList = ['user.json',
#                 'business.json', 
#                 'review.json']

# json_to_csv('data/', fileNameList, createSample=True)

In [341]:
# df_business = pd.read_json('data/business.json',lines=True)

# df_business.dropna(inplace=True, subset = ['categories'], axis=0)

# df_business.loc[df_business['categories'].str.contains('Restaurants')]

# df_business['categories'].value_counts()

In [342]:
#df_user = pd.read_csv('data/user.csv', nrows = 100)

In [297]:
os.listdir('data/')

['filtered_reviews.csv',
 '.DS_Store',
 'business.json',
 'review_sample.csv',
 'business.csv',
 'user.json',
 'review.csv',
 'user.csv',
 'business_sample.csv',
 'user_sample.csv',
 'review.json',
 'restaurants.csv']

In [298]:
# df_review =  pd.read_csv('data/review.csv', usecols = ['business_id', 'user_id', 'stars'])



# user_cnts = df_review['user_id'].value_counts()
# top_users = user_cnts.loc[user_cnts>2].index
# df_review = df_review.loc[df_review['user_id'].isin(top_users)]
# df_review.to_csv('data/filtered_reviews.csv')

In [299]:
# reading filterdreview 

df_review = pd.read_csv('data/filtered_reviews.csv', index_col=0)

In [300]:
# df_review

# df_review['business_id'].value_counts().clip(lower=0,upper=10).hist()

# df_review['user_id'].value_counts().clip(lower=0,upper=10).hist()

# df_review.head()

In [301]:
# def create_matrix(df_review, n_users, n_items):
#     matrix = np.zeros((n_users, n_items))
    
#     for line in df.interlupes():
        
#         matrix[line[1]-1, line[2]-1] = line [3]
        
#     return matrix


In [302]:
# R_df = df_review.pivot(index = 'user_id', columns ='business_id', values = 'stars').fillna(0)
# R_df.head()



In [303]:
# from surprise import SVD
# from surprise import Dataset
# from surprise import NormalPredictor
# from surprise import BaselineOnly
# from surprise import KNNBasic
# from surprise import Reader
# from surprise.model_selection import cross_validate
# from surprise import NMF


In [304]:
def build_fmap_invmap(ser):
    uni_ele = ser.unique()
    fmap = {v:i for i, v in enumerate(uni_ele)}
    invmap = {i:v for i, v in enumerate(uni_ele)}
    return fmap, invmap

In [305]:
df_review.shape

(5284463, 3)

In [306]:
df_review = df_review.head(100000)

In [307]:
bus_fmap, bus_invmap = build_fmap_invmap(df_review['business_id'])
u_fmap, u_invmap = build_fmap_invmap(df_review['user_id'])

In [308]:
u_invmap[
    u_fmap[df_review['user_id'][0]]
  ], df_review['user_id'][0]




('hG7b0MtEbXx5QzbzE6C_VA', 'hG7b0MtEbXx5QzbzE6C_VA')

In [309]:
bus_invmap[bus_fmap[df_review['business_id'].iloc[20]]], df_review['business_id'][0]



('hd5xm20tfSa70-6UqD9-bg', 'ujmEBvifdJM6h6RLv4wQIg')

In [310]:
df_review['business_id'] = df_review['business_id'].map(bus_fmap)

In [311]:
df_review['user_id'] = df_review['user_id'].map(u_fmap)

In [312]:
df_review.head()

,business_id,stars,user_id
0,0,1.0,0
2,1,5.0,1
3,2,5.0,2
6,3,3.0,3
7,4,1.0,4


In [317]:
n_users, n_bus = df_review['user_id'].nunique(), df_review['business_id'].nunique()

In [318]:
n_dim = 5

In [319]:
user_vector_raw = tf.Variable(tf.random_uniform([n_users, n_dim], minval = -1., maxval = 1.))
bus_vector_raw = tf.Variable(tf.random_uniform([n_bus, n_dim], minval = -1., maxval = 1.))

In [320]:
user_vector = tf.tanh(user_vector_raw)
bus_vector = tf.tanh(bus_vector_raw)

In [321]:
user_vector_raw.shape, bus_vector_raw.shape

(TensorShape([Dimension(72254), Dimension(5)]),
 TensorShape([Dimension(13589), Dimension(5)]))

In [322]:
users = tf.placeholder(tf.int32, shape=(None))
businesses = tf.placeholder(tf.int32, shape=(None))
ratings = tf.placeholder(tf.float32, shape=(None))

UserSampled = tf.nn.embedding_lookup(user_vector, users)
BusinessSampled = tf.nn.embedding_lookup(bus_vector, businesses)
UserSampled.set_shape([None, n_dim])
BusinessSampled.set_shape([None, n_dim])

# input tensors for products, users, ratings

In [323]:
estimatedaffinitiesraw = tf.reduce_sum(UserSampled * BusinessSampled, 1)
estimatedaffinities = tf.sigmoid(estimatedaffinitiesraw)*5

In [324]:
loss = tf.reduce_sum(tf.square(estimatedaffinities - ratings))
opt = tf.train.RMSPropOptimizer(learning_rate=.1).minimize(loss)

In [325]:
sess = tf.Session()


In [326]:
rows = np.random.choice(df_review.shape[0], 64)

In [327]:
sess.run(tf.global_variables_initializer())

In [328]:
for i in range(10000):
    rows = np.random.choice(df_review.shape[0], 64)
    dfrows = df_review.iloc[rows]
    fd = {users:dfrows['user_id'].values,
         businesses:dfrows['business_id'].values,
         ratings:dfrows['stars'].values}
    _, l2loss = sess.run([opt, loss], fd)
    if i % 1000 == 0:
        print(l2loss)

280.5542
242.0506
127.281586
97.87049
70.841095
44.41521
66.671135
49.77986
45.456406
28.924908


In [329]:
user_values, bus_values = sess.run([user_vector, bus_vector])

In [330]:
uid = 115
u_invmap[uid]

'ri7itn7-CdpsaPxTToK5cQ'

In [331]:
bname = 'cHdJXLlKNWixBXpDwEGb_A'
bid = bus_fmap[bname]

In [332]:
bid

12

In [333]:
chipotle = bus_values[bid]

In [334]:
chipotle

array([-0.9999995 , -0.7207982 , -0.9999928 ,  0.99999905,  0.9694086 ],
      dtype=float32)

In [335]:
japaneselover = user_values[uid]

In [336]:
bus_values

array([[-0.8646391 ,  0.99999917,  0.9999984 , -0.928239  , -0.9983322 ],
       [-0.99998283,  0.98596376,  0.07949587, -0.99999475,  0.45583826],
       [-0.9995487 ,  0.94259447, -0.9912963 , -0.9978772 ,  0.94698477],
       ...,
       [-0.26906186, -0.8860855 , -0.77044463, -0.9242717 ,  0.52560014],
       [-0.9905625 , -0.9905412 , -0.99279475, -0.99459225,  0.99750453],
       [ 0.6112084 , -0.6120856 ,  0.82910144,  0.54135585,  0.7058714 ]],
      dtype=float32)

In [337]:
np.square(bus_values - japaneselover[None,:]).sum(1).argsort()

array([ 8092, 10561,  3252, ...,  2516,  1902,   855])

In [276]:
np.square(bus_values - chipotle[None,:]).sum(1).argsort()

array([   12, 11899,    91, ...,  1652,   663,  1805])

In [277]:
def closest_businesses_to(business = None, user = None, df = None):
    if business is not None:
        target = bus_values[bus_fmap[business]]
    if user is not None:
        target = user_values[u_fmap[user]]
    if df is None:
        df = bus_values
    best_restaurants = np.square(df - target[None,:]).sum(1).argsort()
    return best_restaurants

In [278]:
midtown_japanese_restaurants = bus_values[:30,:]

In [279]:
closest_businesses_to(business = 'cHdJXLlKNWixBXpDwEGb_A')

array([   12, 11899,    91, ...,  1652,   663,  1805])

In [280]:
closest_businesses_to(user = 'ri7itn7-CdpsaPxTToK5cQ')

array([ 9872,   523, 11100, ...,  2571,  1882,  1671])

In [281]:
closest_businesses_to(user = 'ri7itn7-CdpsaPxTToK5cQ', df = midtown_japanese_restaurants)

array([ 4, 18,  3,  9, 21, 26, 13, 11, 24, 27, 12,  8, 22, 10, 28,  6,  5,
       15, 16,  1, 25,  7, 17,  0,  2, 23, 14, 19, 29, 20])

In [284]:
# from surprise.model_selection import train_test_split


In [285]:
# reader = Reader(rating_scale=(0, 5))
# data = Dataset.load_from_df(df_review[[
#     'business_id', 'stars', 'user_id']], reader)
#data2 = Dataset.load_builtin('ml-100k')

In [ ]:
# trainset = data.build_full_trainset()

In [ ]:
#trainset, testset = train_test_split(data, test_size=.25)

In [ ]:
# model = NMF(n_factors = 5, n_epochs = 2)

In [ ]:
# modeltrained = model.fit(trainset)

In [ ]:
# sim = modeltrained.compute_similarities()

In [ ]:
# sim

In [ ]:
# import tensorflow as tf

In [ ]:
# 3

In [ ]:
# ??modeltrained.compute_similarities

In [ ]:
# ??NMF

In [ ]:
# benchmark = []
# # Iterate over all algorithms
# for algorithm in [NMF()]:
#     # Perform cross validation
#     results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
#     # Get results & append algorithm name
#     tmp = pd.DataFrame.from_dict(results).mean(axis=0)
#     tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
#     benchmark.append(tmp)
    
# pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')    

In [ ]:
#df_review.fillna(df_review.mean(), inplace=True)

In [ ]:
#print(df_review.dtypes)